<a href="https://colab.research.google.com/github/EdH66/CMAP-Analysis/blob/main/CMAP_Drug_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.metrics import jaccard_score
import numpy as np


In [6]:
# Read the CSV file
url = '/content/drive/My Drive/Colab Notebooks/CMAP Drug Prediction/CMAP_Jaquard_test.csv'  # Replace with the URL of your CSV file
df = pd.read_csv(url)



Jaquard similarity

In [7]:
# Function to calculate Jaccard similarity
def calculate_jaccard_similarity(series1, series2):
    set1 = set(series1.dropna())
    set2 = set(series2.dropna())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    if not union:  # Prevent division by zero
        return np.nan
    return len(intersection) / len(union)



In [10]:


# Extract columns
col1 = df['8 weeks']
col3 = df['12 weeks']
col5 = df['16 weeks']

# Calculate Jaccard similarities
similarity_13 = calculate_jaccard_similarity(col1, col3)
similarity_15 = calculate_jaccard_similarity(col1, col5)
similarity_35 = calculate_jaccard_similarity(col3, col5)

# Print results
print(f"Jaccard Similarity between Week 8 and Week 12: {similarity_13}")
print(f"Jaccard Similarity between Week 8 and Week 16: {similarity_15}")
print(f"Jaccard Similarity between Week 12 and Week 16: {similarity_35}")

Jaccard Similarity between Week 8 and Week 12: 0.39485087942900843
Jaccard Similarity between Week 8 and Week 16: 0.16097809475292918
Jaccard Similarity between Week 12 and Week 16: 0.11102106969205834


Attempt at weighted similarity

In [15]:
def custom_similarity(df, col_drug1, col_ncs1, col_drug2, col_ncs2):
    # Ensure NCS columns are numeric
    df[col_ncs1] = pd.to_numeric(df[col_ncs1], errors='coerce')
    df[col_ncs2] = pd.to_numeric(df[col_ncs2], errors='coerce')

    # Drop rows where either drug name or NCS is NaN
    df = df.dropna(subset=[col_drug1, col_ncs1, col_drug2, col_ncs2])

    # Creating dictionaries to pair drugs with their NCS values
    drugs_at_time1 = dict(zip(df[col_drug1], df[col_ncs1]))
    drugs_at_time2 = dict(zip(df[col_drug2], df[col_ncs2]))

    # Identifying common and unique drugs
    common_drugs = set(drugs_at_time1.keys()).intersection(set(drugs_at_time2.keys()))
    all_drugs = set(drugs_at_time1.keys()).union(set(drugs_at_time2.keys()))

    # Calculating weighted intersection and union
    intersection = sum(min(drugs_at_time1.get(drug, 0), drugs_at_time2.get(drug, 0)) for drug in all_drugs)
    union = sum(max(drugs_at_time1.get(drug, 0), drugs_at_time2.get(drug, 0)) for drug in all_drugs)

    # Computing similarity
    return intersection / union if union != 0 else 0

# Example usage
similarity_8_12 = custom_similarity(df, '8 weeks', 'NCS', '12 weeks', 'NCS')
similarity_8_16 = custom_similarity(df, '8 weeks', 'NCS', '16 weeks', 'NCS')
similarity_12_16 = custom_similarity(df, '12 weeks', 'NCS', '16 weeks', 'NCS')

print(f"Similarity between Week 8 and Week 12: {similarity_8_12}")
print(f"Similarity between Week 8 and Week 16: {similarity_8_16}")
print(f"Similarity between Week 12 and Week 16: {similarity_12_16}")



Similarity between Week 8 and Week 12: 3.143496349250033
Similarity between Week 8 and Week 16: 7.7715833689922
Similarity between Week 12 and Week 16: 13.552498471211056
